### Step 1: Import essential libraries and create a Spark Session

In [18]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f

spark = SparkSession.builder.getOrCreate()

### Step 2: Read the input file

In [19]:
df = spark.read.parquet("../../Datasets/shapes.parquet")

In [33]:
df.show(truncate=False)

+--------+--------------------------------------------+
|shape_id|vertices                                    |
+--------+--------------------------------------------+
|Shape_0 |[[35, 28], [43, 28], [43, 32], [35, 32]]    |
|Shape_1 |[[44, 16], [53, 16], [53, 20], [44, 20]]    |
|Shape_2 |[[67, 84], [76, 84], [76, 92], [67, 92]]    |
|Shape_3 |[[29, 37], [37, 37], [37, 45], [29, 45]]    |
|Shape_4 |[[51, 79], [54, 79], [54, 88], [51, 88]]    |
|Shape_5 |[[55, 33], [62, 33], [62, 39], [55, 39]]    |
|Shape_6 |[[17, 88], [21, 88], [21, 97], [17, 97]]    |
|Shape_7 |[[1, 2], [10, 2], [10, 7], [1, 7]]          |
|Shape_8 |[[65, 14], [69, 14], [69, 20], [65, 20]]    |
|Shape_9 |[[12, 48], [19, 48], [19, 51], [12, 51]]    |
|Shape_10|[[97, 20], [100, 20], [100, 26], [97, 26]]  |
|Shape_11|[[7, 38], [15, 38], [15, 45], [7, 45]]      |
|Shape_12|[[69, 65], [78, 65], [78, 68], [69, 68]]    |
|Shape_13|[[35, 8], [42, 8], [42, 18], [35, 18]]      |
|Shape_14|[[96, 35], [100, 35], [100, 38], [96, 

Looks like this dataset contains all rectangles, let's verify this hypothesis.

In [20]:
df_sorted = df.withColumn("vertices", f.expr("array_sort(vertices)"))
df_sorted.show(truncate=False)

+--------+--------------------------------------------+
|shape_id|vertices                                    |
+--------+--------------------------------------------+
|Shape_0 |[[35, 28], [35, 32], [43, 28], [43, 32]]    |
|Shape_1 |[[44, 16], [44, 20], [53, 16], [53, 20]]    |
|Shape_2 |[[67, 84], [67, 92], [76, 84], [76, 92]]    |
|Shape_3 |[[29, 37], [29, 45], [37, 37], [37, 45]]    |
|Shape_4 |[[51, 79], [51, 88], [54, 79], [54, 88]]    |
|Shape_5 |[[55, 33], [55, 39], [62, 33], [62, 39]]    |
|Shape_6 |[[17, 88], [17, 97], [21, 88], [21, 97]]    |
|Shape_7 |[[1, 2], [1, 7], [10, 2], [10, 7]]          |
|Shape_8 |[[65, 14], [65, 20], [69, 14], [69, 20]]    |
|Shape_9 |[[12, 48], [12, 51], [19, 48], [19, 51]]    |
|Shape_10|[[97, 20], [97, 26], [100, 20], [100, 26]]  |
|Shape_11|[[7, 38], [7, 45], [15, 38], [15, 45]]      |
|Shape_12|[[69, 65], [69, 68], [78, 65], [78, 68]]    |
|Shape_13|[[35, 8], [35, 18], [42, 8], [42, 18]]      |
|Shape_14|[[96, 35], [96, 38], [100, 35], [100, 

In [36]:
df_rect_check = df_sorted \
    .withColumn("x0", f.col("vertices")[0][0]) \
    .withColumn("y0", f.col("vertices")[0][1]) \
    .withColumn("x1", f.col("vertices")[1][0]) \
    .withColumn("y1", f.col("vertices")[1][1]) \
    .withColumn("x2", f.col("vertices")[2][0]) \
    .withColumn("y2", f.col("vertices")[2][1]) \
    .withColumn("x3", f.col("vertices")[3][0]) \
    .withColumn("y3", f.col("vertices")[3][1]) \
    .withColumn("is_rect", 
        (f.col("x0") == f.col("x1")) &
        (f.col("y0") == f.col("y2")) &
        (f.col("y1") == f.col("y3")) &
        (f.col("x3") == f.col("x2"))
    )

df_rect_check.filter(~f.col("is_rect")).show(truncate=False)


+--------+--------+---+---+---+---+---+---+---+---+-------+
|shape_id|vertices|x0 |y0 |x1 |y1 |x2 |y2 |x3 |y3 |is_rect|
+--------+--------+---+---+---+---+---+---+---+---+-------+
+--------+--------+---+---+---+---+---+---+---+---+-------+



Based on this result, we can narrow the problem down to detecting overlapping upright rectangles.

### Step 3: Computes the bounding box (min/max x and y) for each shape based on its vertices.

In [21]:
vertices_df = df.withColumn("v", f.explode(f.col("vertices"))) \
    .withColumn("x", f.col("v")[0]) \
    .withColumn("y", f.col("v")[1]) \
    .groupBy("shape_id") \
    .agg(
        f.expr("min(x)").alias("x_min"),
        f.expr("max(x)").alias("x_max"),
        f.expr("min(y)").alias("y_min"),
        f.expr("max(y)").alias("y_max")
    )

In [22]:
vertices_df.show(truncate=False)

+---------+-----+-----+-----+-----+
|shape_id |x_min|x_max|y_min|y_max|
+---------+-----+-----+-----+-----+
|Shape_245|37   |41   |60   |65   |
|Shape_290|34   |40   |98   |102  |
|Shape_465|71   |80   |63   |68   |
|Shape_473|33   |43   |24   |32   |
|Shape_490|33   |43   |26   |30   |
|Shape_739|43   |50   |49   |59   |
|Shape_763|19   |25   |44   |54   |
|Shape_834|69   |75   |29   |35   |
|Shape_902|82   |86   |19   |23   |
|Shape_104|51   |54   |36   |44   |
|Shape_472|44   |49   |82   |87   |
|Shape_476|54   |64   |80   |90   |
|Shape_478|97   |106  |69   |73   |
|Shape_564|75   |81   |2    |11   |
|Shape_762|31   |34   |82   |91   |
|Shape_833|20   |27   |93   |100  |
|Shape_151|75   |84   |70   |75   |
|Shape_306|54   |59   |73   |82   |
|Shape_595|0    |9    |28   |32   |
|Shape_765|54   |61   |56   |65   |
+---------+-----+-----+-----+-----+
only showing top 20 rows



### Step 4: Perform a self cross-join to generate shape pairs

In [23]:
df1 = vertices_df.alias("a")
df2 = vertices_df.alias("b")
joined = df1.crossJoin(df2).filter(f.col("a.shape_id") < f.col("b.shape_id"))

### Step 5: Check for overlapping bounding box

In [24]:
overlapping = joined.filter(
    (f.col("a.x_min") < f.col("b.x_max")) & (f.col("a.x_max") > f.col("b.x_min")) &
    (f.col("a.y_min") < f.col("b.y_max")) & (f.col("a.y_max") > f.col("b.y_min"))
)

### Step 6: Choose the overlapping pair

In [29]:
result = overlapping.select(
    f.col("a.shape_id").alias("shape_1"),
    f.col("b.shape_id").alias("shape_2")
)

result = result.withColumn("shape_1", f.regexp_replace(f.col("shape_1"), "Shape_", "").cast("int")) \
                     .withColumn("shape_2", f.regexp_replace(f.col("shape_2"), "Shape_", "").cast("int"))

result = result.orderBy("shape_1", "shape_2")

In [30]:
result.show(truncate=False)

+-------+-------+
|shape_1|shape_2|
+-------+-------+
|0      |41     |
|0      |143    |
|0      |198    |
|0      |430    |
|0      |473    |
|0      |490    |
|0      |496    |
|0      |605    |
|0      |637    |
|0      |729    |
|0      |818    |
|1      |118    |
|1      |160    |
|1      |186    |
|1      |220    |
|1      |247    |
|1      |256    |
|1      |377    |
|1      |444    |
|1      |809    |
+-------+-------+
only showing top 20 rows



### Step 7: Verify the output

Read the output from source code directory

In [31]:
main_df = spark.read.csv('../../src/Task_2.3/output.csv',header= True)

Check whether the dataframes are identical

In [ ]:
# Rows in main_df but not in result
diff1 = main_df.subtract(result)

# Rows in result but not in main_df
diff2 = result.subtract(main_df)

# Union 2 the different rows
differences = diff1.union(diff2)

# Print the result
differences.show()

print(f"Number of different rows: {differences.count()}")

+-------+-------+
|shape_1|shape_2|
+-------+-------+
+-------+-------+



Number of rows different: 0
